## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.5 MB/s 
     |████████████████████████████████| 7.6 MB 81.5 MB/s 
     |████████████████████████████████| 182 kB 102.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 15.5 MB/s 
     |████████████████████████████████| 212 kB 81.5 MB/s 
     |████████████████████████████████| 132 kB 88.1 MB/s 
     |████████████████████████████████| 127 kB 82.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 7.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [2]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATSC
from InstructABSA.data_prep import ModelReadyData

In [4]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [5]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()
res_tr_df = model_data.extract_rowwise_aspect_polarity(res_tr_df, on='aspectTerms', by="term", min_val=1)
lap_tr_df = model_data.extract_rowwise_aspect_polarity(lap_tr_df, on='aspectTerms', by="term", min_val=1)
res_te_df = model_data.extract_rowwise_aspect_polarity(res_te_df, on='aspectTerms', by="term", min_val=1)
lap_te_df = model_data.extract_rowwise_aspect_polarity(lap_te_df, on='aspectTerms', by="term", min_val=1)

 # Get the input text into the required format
bos_instruction_lap = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none. 
Positive example 1- input: I charge it at night and skip taking the cord with me because of the good battery life. Aspect: battery life. output: positive. 
Positive example 2- input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!. Aspect: Photobooth. output: positive. 
Now complete the following example-
input: """

bos_instruction_res = """Definition: The output will be 'positive' if the aspect identified in the sentence contains a positive sentiment. If the sentiment of the identified aspect in the input is negative the answer will be 'negative'. 
Otherwise, the output should be 'neutral'. For aspects which are classified as noaspectterm, the sentiment is none. 
Positive example 1- input: With the great variety on the menu , I eat here often and never get bored. Aspect: menu. output: positive. 
Positive example 2- input: Great food, good size menu, great service and an unpretensious setting. Aspect: food. output: positive. 
Now complete the following example-
input: """
delim_instruction = ' Aspect: '
eos_insrtuction = '. Output:'

res_tr_df = model_data.create_data_in_atsc_format(res_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_tr_df = model_data.create_data_in_atsc_format(lap_tr_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)
res_te_df = model_data.create_data_in_atsc_format(res_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_res, delim_instruction, eos_insrtuction)
lap_te_df = model_data.create_data_in_atsc_format(lap_te_df, 'raw_text', 'aspect', 'labels', bos_instruction_lap, delim_instruction, eos_insrtuction)

## 10% Training Data

In [8]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.1)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

In [ ]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '10pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '10pc_' + id_te_filename), index = False)

Train accuracy:  86.558516801854
Test accuracy:  81.22881355932203


## 20% Training Data

In [6]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.2)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [7]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1726
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 432
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.309338
2,No log,0.217863
3,No log,0.242022
4,No log,0.219781


***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-d

In [8]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1726
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


***** Running Prediction *****
  Num examples = 2360
  Batch size = 16


In [9]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '20pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '20pc_' + id_te_filename), index = False)

Train accuracy:  88.00695249130939
Test accuracy:  85.12711864406779


## 50% Training Data

In [6]:
# Experimentation
experiment_id = 'combined100'
experiment_name = 'combined100_instruct'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'

# Create T5 utils object
t5_exp = T5ATSC(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df, sample_size=0.5)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATSC')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
dataset, tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/677 [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [7]:
# Train Model
model_trainer = t5_exp.train(tokenized_dataset, **training_args)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 4314
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1080
  Number of trainable parameters = 247534848
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.373444
2,0.345700,0.196503
3,0.345700,0.185866
4,0.188400,0.190243


***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2360
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-d

In [8]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = tokenized_dataset, sample_set = 'validation')

The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 4314
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATSC/allenai/tk-instruct-base-def-pos-combined100_instruct/checkpoints


***** Running Prediction *****
  Num examples = 2360
  Batch size = 16


In [9]:
# Add new column in the respective dataframes
tr_df = pd.DataFrame(dataset['train'])
te_df = pd.DataFrame(dataset['validation'])

tr_df['pred_labels'] =  tr_pred_labels
te_df['pred_labels'] =  te_pred_labels

# Metrics
print('Train accuracy: ', tr_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(tr_df))
print('Test accuracy: ', te_df[['labels', 'pred_labels']].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
tr_df.to_csv(os.path.join(dump_path, '50pc_' + id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
te_df.to_csv(os.path.join(dump_path, '50pc_' + id_te_filename), index = False)

Train accuracy:  90.77422345850718
Test accuracy:  86.73728813559322
